<a href="https://colab.research.google.com/github/nstsj/python_for_CL/blob/master/morpho-analysis/morphology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Сегодня мы поговорим о морфологических анализаторах для русского языка. Важно отметить, что (в основном) для каждого естественного языка нужен свой морфоанализатор.

* шаг 1: установим пайморфи и майстем

In [2]:
!pip install pymystem3==0.1.10
!pip install pymorphy2[fast]

  Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [5]:
# импорты
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

# сохраняем класс в переменную
mystem = Mystem() 
morph = MorphAnalyzer() 

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [0]:
# сэмпл-текст, на котором все будем пробовать
text = """Система состоит из камеры и программного обеспечения, которое анализирует фотографию.
 Суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных"""

## нормализация

давайте приведем текст к нижнему регистру

In [117]:
text = text.lower()
text

'система состоит из камеры и программного обеспечения, которое анализирует фотографию.\n суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных'

## токенизация

иногда текст нужно предварительно разбить на токены, отдельные элементы. Токенизаторов много, каждый делает это, руководствуясь своими правилами

* токенизация нужна, если морофоанализатор не умеет токенизировать сам (Mystem умеет, pymorphy не умеет)

In [0]:
# попробуем токенизацию от модуля gensim
from gensim.utils import tokenize

list(tokenize(text))

In [0]:
#  еще можно токенизировать функциями из модуля nltk

from nltk.tokenize import word_tokenize, wordpunct_tokenize # попробуем две функции

In [120]:
wordpunct_tokenize("O'Brian, Ростов-на-Дону, обычноеслово")

['O', "'", 'Brian', ',', 'Ростов', '-', 'на', '-', 'Дону', ',', 'обычноеслово']

In [122]:
import nltk
nltk.download('punkt')

word_tokenize("O'Brian, Ростов-на-Дону, обычноеслово")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


["O'Brian", ',', 'Ростов-на-Дону', ',', 'обычноеслово']

## лемматизация и морфоанализ

### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст.

In [79]:
# сначала лемматизируем слова методом .lemmatize()
print(text)
print(mystem.lemmatize(text))


система состоит из камеры и программного обеспечения, которое анализирует фотографию.
 Суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных
['система', ' ', 'состоять', ' ', 'из', ' ', 'камера', ' ', 'и', ' ', 'программный', ' ', 'обеспечение', ', ', 'который', ' ', 'анализировать', ' ', 'фотография', '.', '\n', ' ', 'суть', ' ', 'технология', ' — ', 'сопоставление', ' ', 'лицо', ', ', 'попадать', ' ', 'в', ' ', 'объектив', ', ', 'с', ' ', 'изображение', ' ', 'из', ' ', 'база', ' ', 'данные', '\n']


In [0]:
# метод .analyze() даст грамматическую информацию о словах
words_analized = mystem.analyze(text)

этот метод возвращает список словарей


каждый словарь имеет либо одно поле 'text' (когда попался пробел или пунктуация) или text и analysis

* в analysis снова список словарей с вариантами разбора (первый самый вероятный)
* поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
* analysis - может быть пустым списком

In [0]:
words_analized # посмотрим на анализ и теги

In [126]:
# сделаем все красиво с индексами и доступом по ключам

print('Слово - ', words_analized[0]['text'])
print('Разбор слова - ', words_analized[0]['analysis'][0])
print('Лемма слова - ', words_analized[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[0]['analysis'][0]['gr'])

Слово -  система
Разбор слова -  {'lex': 'система', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}
Лемма слова -  система
Грамматическая информация слова -  S,жен,неод=им,ед


In [75]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

['система',
 'состоять',
 'из',
 'камера',
 'и',
 'программный',
 'обеспечение',
 'который',
 'анализировать',
 'фотография']

In [0]:
# то же самое, что в предыдущей ячейке, но циклом
res = []
for parse in words_analized:
    if parse.get('analysis'):
        res.append(parse['analysis'][0]['lex'])

res[:10]

#### дополнительные возможности Mystem

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/

После этого сохранить текст в файл.

In [0]:
f = open('text.txt', 'w')
f.write(text)
f.close()

Из командной строки или из питона запустить майстем на нашем файле

In [0]:
# про параметры почитайте в ./mystem -h
os.system(' ./mystem -iscd --format json text.txt text_parsed.txt')

В целевом файле теперь лежит разобранный текст в jsonlines (json на каждой строчке)

In [0]:
t = [json.loads(line) for line in open('text_parsed.txt')]

Каждый объект в этом списке - параграф. Каждый параграф на предложения можно разбив по тегу '//s'

Ещё так вызывать майстем может понадобиться, если важна скорость.

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### Pymorphy

Pymorphy - открытый и развивается (можно поучаствовать на гитхабе)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

Попробуйте сразу установить быструю версию (pip install pymorphy2[fast])

* [документация pymorphy](https://pythonhosted.org/pymorphy/)

У него нет втстроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. Это лучше стемминга, но хуже майстема.

In [0]:
# основная функция - pymorphy.parse
words_analized = [morph.parse(token) for token in word_tokenize(text)]

In [0]:
words_analized

In [129]:
morph.parse("печь") # пример с морфологической неоднозначностью (форма слова может пониматься по-разному)

[Parse(word='печь', tag=OpencorporaTag('INFN,impf,tran'), normal_form='печь', score=0.666666, methods_stack=((<DictionaryAnalyzer>, 'печь', 2352, 0),)),
 Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='печь', score=0.166666, methods_stack=((<DictionaryAnalyzer>, 'печь', 2131, 0),)),
 Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='печь', score=0.166666, methods_stack=((<DictionaryAnalyzer>, 'печь', 2131, 3),))]

Она похожа на analyze в майстеме только возвращает список объектов Parse
* Первый в списке - самый вероятный разбор (у каждого есть score)
* Информация достается через атрибут (Parse.word - например)
* Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
части речи или другие категории

In [130]:
# сделаем красиво и тут
print('Первое слово - ', words_analized[0][0].word)
print('Разбор первого слова - ', words_analized[0][0])
print('Лемма первого слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analized[0][0].tag)
print('Часть речи первого слова - ', words_analized[0][0].tag.POS)
print('Род первого слова - ', words_analized[0][0].tag.gender)
print('Число первого слова - ', words_analized[0][0].tag.number)
print('Падеж первого слова - ', words_analized[0][0].tag.case)

Первое слово -  система
Разбор первого слова -  Parse(word='система', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='система', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'система', 55, 0),))
Лемма первого слова -  система
Грамматическая информация первого слова -  NOUN,inan,femn sing,nomn
Часть речи первого слова -  NOUN
Род первого слова -  femn
Число первого слова -  sing
Падеж первого слова -  nomn


### что можно дальше

Pymorphy и Mystem - не единственные морфоанализаторы для русского языка. Можно, например, посмотреть на [RNNmorph](https://github.com/IlyaGusev/rnnmorph) и [deeppavlov](http://docs.deeppavlov.ai/en/master/features/models/morphotagger.html).

А еще есть исследование, где сравнивали морфоанализаторы для русского  ([краткая версия](http://web-corpora.net/wsgi/mystemplus.wsgi/mystemplus/compare_table/), [статья](http://www.dialog-21.ru/media/3473/dereza.pdf))

 